# Modules

In [ ]:
try:
    from google.colab import drive
    drive.mount('/content/gdrive')
    dataset_Path = '/content/gdrive/MyDrive/Datasets/'
    model_Path = '/content/gdrive/MyDrive/models/'
except:
    dataset_Path = '/home/bilz/air/Datasets/'
    model_Path = '/home/bilz/air/models/'
    

dataset_Path_Full = dataset_Path + 'trainingTestingData.csv'
DATA_PATH = dataset_Path_Full
print(DATA_PATH)

In [1]:
%load_ext autoreload
%autoreload 2

import sys

import matplotlib.pyplot as plt
import pandas as pd
import torch
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.manifold import TSNE
from sklearn.metrics import (ConfusionMatrixDisplay, classification_report,
                             confusion_matrix)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.optim import Adam
from torch.utils.data import DataLoader

sys.path.append("../")

from scarf.loss import NTXent
from scarf.model import SCARF

from example.dataset import ExampleDataset
from example.utils import dataset_embeddings, fix_seed, train_epoch

seed = 1234
fix_seed(seed)

/home/bilz/miniconda3/envs/pt/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Data

In [2]:
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer(as_frame=True)
df = data.frame  # Accessing the DataFrame from the loaded data

# Printing the first 10 items using the head() method
df.head(10)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,target
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.30010,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.08690,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.19740,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.24140,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.19800,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0
5,12.45,15.70,82.57,477.1,0.12780,0.17000,0.15780,0.08089,0.2087,0.07613,...,23.75,103.40,741.6,0.1791,0.5249,0.5355,0.1741,0.3985,0.12440,0
6,18.25,19.98,119.60,1040.0,0.09463,0.10900,0.11270,0.07400,0.1794,0.05742,...,27.66,153.20,1606.0,0.1442,0.2576,0.3784,0.1932,0.3063,0.08368,0
7,13.71,20.83,90.20,577.9,0.11890,0.16450,0.09366,0.05985,0.2196,0.07451,...,28.14,110.60,897.0,0.1654,0.3682,0.2678,0.1556,0.3196,0.11510,0
8,13.00,21.82,87.50,519.8,0.12730,0.19320,0.18590,0.09353,0.2350,0.07389,...,30.73,106.20,739.3,0.1703,0.5401,0.5390,0.2060,0.4378,0.10720,0
9,12.46,24.04,83.97,475.9,0.11860,0.23960,0.22730,0.08543,0.2030,0.08243,...,40.68,97.65,711.4,0.1853,1.0580,1.1050,0.2210,0.4366,0.20750,0


In [25]:
import pandas as pd

# Load the existing CSV file
file_path = "/home/bilz/Datasets/CIOD/baselineIOD.csv"
data = pd.read_csv(file_path)

# Printing the first 10 items using the head() method
data.head(10)




,RSRP,RSRQ,Light,Mag,Acc,Sound,Proximity,Daytime,New_Recording,IO
0,-88.0,-15.0,1.32,43.860838,0.162125,30.767,5.0,0,0,1.0
1,-88.0,-15.0,1.32,42.138897,0.232108,29.067,5.0,0,0,1.0
2,-88.0,-15.0,1.32,43.117873,0.076151,25.418,5.0,0,0,1.0
3,-88.0,-15.0,1.32,42.454490,0.698630,22.555,5.0,0,0,1.0
4,-88.0,-15.0,1.32,44.450612,0.701351,24.986,5.0,0,0,1.0
5,-88.0,-15.0,1.32,43.958620,1.426823,29.059,5.0,0,0,1.0
6,-88.0,-15.0,1.32,42.846911,1.039526,24.823,5.0,0,0,1.0
7,-88.0,-15.0,1.32,44.122328,0.323144,22.341,5.0,0,0,1.0
8,-88.0,-15.0,1.32,42.881168,0.547964,24.725,5.0,0,0,1.0
9,-88.0,-15.0,1.32,44.126031,0.492282,23.895,5.0,0,0,1.0


In [9]:
# Assuming `mydataset` contains the specified columns
mydataset = pd.DataFrame({
    "RSRP": [...],  #  RSRP data
    "RSRQ": [...],  #  RSRQ data
    "Light": [...],  #  Light data
    "Mag": [...],  #  Mag data
    "Acc": [...],  #  Acc data
    "Sound": [...],  #  Sound data
    "Proximity": [...],  #  Proximity data
    "Daytime": [...],  #  Daytime data
    "New_Recording": [...],  #  New_Recording data
    "IO": [...]  #  IO data
})

# Extract specific columns for data and target
data = mydataset[["RSRP", "RSRQ", "Light", "Mag", "Acc", "Sound", "Proximity", "Daytime", "New_Recording"]]
target = mydataset["IO"]

In [23]:
import os
from functions import *

DATA_PATH = '/home/bilz/Datasets/CIOD/baselineIOD.csv'
print(DATA_PATH)

# Check if the path exists (file or folder)
if os.path.exists(DATA_PATH):
    print(f"{DATA_PATH} exists.")
    if os.path.isfile(DATA_PATH):
        print(f"It is a file.")
    elif os.path.isdir(DATA_PATH):
        print(f"It is a directory.")
else:
    print(f"{DATA_PATH} does not exist.")


/home/bilz/Datasets/CIOD/baselineIOD.csv
/home/bilz/Datasets/CIOD/baselineIOD.csv exists.
It is a file.


In [27]:
# Preprocess data
df, data = preprocessData(DATA_PATH)

NameError: name 'DATA_PATH' is not defined

In [14]:
# Split the data into train and test sets
train_data, test_data, train_target, test_target = train_test_split(
    data,
    target,
    test_size=0.2,
    stratify=target,  # Stratify the split based on the target variable
    random_state=seed
)

# Preprocess
constant_cols = [c for c in train_data.columns if train_data[c].nunique() == 1]
train_data.drop(columns=constant_cols, inplace=True)
test_data.drop(columns=constant_cols, inplace=True)

scaler = StandardScaler()
train_data = pd.DataFrame(scaler.fit_transform(train_data), columns=train_data.columns)
test_data = pd.DataFrame(scaler.transform(test_data), columns=test_data.columns)

# To torch dataset
# Assuming ExampleDataset class handles the torch dataset creation
train_ds = ExampleDataset(
    train_data.to_numpy(),
    train_target.to_numpy(),
    columns=train_data.columns
)
test_ds = ExampleDataset(
    test_data.to_numpy(),
    test_target.to_numpy(),
    columns=test_data.columns
)

print(f"Train set: {train_ds.shape}")
print(f"Test set: {test_ds.shape}")
train_ds.to_dataframe().head()

ValueError: With n_samples=1, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.

In [10]:
#data = datasets.load_breast_cancer(as_frame=True)
#data, target = data["data"], data["IO"]
train_data, test_data, train_target, test_target = train_test_split(
    data,
    target,
    test_size=0.2,
    stratify=target,
    random_state=seed
)

# preprocess
constant_cols = [c for c in train_data.columns if train_data[c].nunique() == 1]
train_data.drop(columns=constant_cols, inplace=True)
test_data.drop(columns=constant_cols, inplace=True)

scaler = StandardScaler()
train_data = pd.DataFrame(scaler.fit_transform(train_data), columns=train_data.columns)
test_data = pd.DataFrame(scaler.transform(test_data), columns=test_data.columns)

# to torch dataset
train_ds = ExampleDataset(
    train_data.to_numpy(),
    train_target.to_numpy(),
    columns=train_data.columns
)
test_ds = ExampleDataset(
    test_data.to_numpy(),
    test_data.to_numpy(),
    columns=test_data.columns
)

print(f"Train set: {train_ds.shape}")
print(f"Test set: {test_ds.shape}")
train_ds.to_dataframe().head()

ValueError: With n_samples=1, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.